## Executive Summary

This project explored data from the Centers of Medicare & Medicaid Services (CMS) on nursing home provider information in order to fit a feed-forward deep neural network machine learning model to predict the binary outcome of whether any given nursing will be cited for abuse. We discovered that by feeding the continuous and boolean features of the average number of residents per day, whether a provider resides in a hospital, whether their most recent health inspection was more than two years ago, whether a provider changed ownership within the last 12 months, the total number of penalties that a nursing home has received from CMS, and the reported nurse aide, LPN, RN, total licensed staffing, and total nurse staffing hours per resident per day, can predict whether a nursing home will be cited for abuse about 66% of the time.

## Introduction

Staffing levels in nursing homes have long been known to be correlative or outright predictive of a variety of outcomes, such as whether residents are victims of abuse (Dicken, 2019), overall low quality of care (Dummit, 2002; Harrington et al., 2016), and even injury to nursing home staff themselves (Trinkoff et al., 2005). These effects are known to exist on political (Blankart et al., 2019), financial (Grabowski et al., 2017), and geographic (Park & Martin, 2018) lines. Adequately addressing the staffing needs of nursing homes in the United States is a major problem with no easy solutions, given the difficulties of working in such settings and the needs of the particular patients who reside within them. 

The purpose of this study was to explore the extent to which the average number of residents per day, whether a provider resides in a hospital, whether their most recent health inspection was more than two years ago, whether a provider changed ownership within the last 12 months, the total number of penalties that a nursing home has received from CMS, and the reported nurse aide, LPN, RN, total licensed staffing, and total nurse staffing hours per resident per day can predict whether a nursing home will be cited for abuse. Addressing staffing needs in nursing homes must begin long before a staff person walks through any particular set of nursing home doors (Jane Banaszak-Holl et al., 2002) and has implications for how nursing home staff are compensated, suggesting changes needed at the federal level for reimbursement rates for nursing home medical professionals (Konetzka et al., 2006). However, other dimensions, such as ownership, setting, and health inspection status, may also be important.

We apply a TensorFlow feed-forward deep neural network model to this predictive exercise.

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

In [2]:
df = pd.read_csv('Provider_Info.csv').set_index('Federal Provider Number')

## Preparation

We'll take a look at our data and figure out what we need to do to prepare it for a TensorFlow DNN.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15491 entries, 45198 to 675792
Data columns (total 85 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Provider Name                                                    15491 non-null  object 
 1   Provider Address                                                 15491 non-null  object 
 2   Provider City                                                    15491 non-null  object 
 3   Provider State                                                   15491 non-null  object 
 4   Provider Zip Code                                                15491 non-null  int64  
 5   Provider Phone Number                                            15491 non-null  int64  
 6   Provider SSA County Code                                         15491 non-null  int64  
 7   Provider County Name                    

For our purposes here, we're only interested in continuous and Boolean features, so we'll filter those out.

In [4]:
df = df.select_dtypes(include=['int64', 'float64', 'bool'])
df.columns.to_list()

['Provider Zip Code',
 'Provider Phone Number',
 'Provider SSA County Code',
 'Number of Certified Beds',
 'Average Number of Residents Per Day',
 'Provider Resides in Hospital',
 'Continuing Care Retirement Community',
 'Cited for Abuse',
 'Most Recent Health Inspection More Than 2 Years Ago',
 'Provider Changed Ownership in Last 12 Months',
 'Overall Rating',
 'Overall Rating Footnote',
 'Health Inspection Rating',
 'Health Inspection Rating Footnote',
 'QM Rating',
 'QM Rating Footnote',
 'Long-Stay QM Rating',
 'Long-Stay QM Rating Footnote',
 'Short-Stay QM Rating',
 'Short-Stay QM Rating Footnote',
 'Staffing Rating',
 'Staffing Rating Footnote',
 'RN Staffing Rating',
 'RN Staffing Rating Footnote',
 'Reported Staffing Footnote',
 'Physical Therapist Staffing Footnote',
 'Reported Nurse Aide Staffing Hours per Resident per Day',
 'Reported LPN Staffing Hours per Resident per Day',
 'Reported RN Staffing Hours per Resident per Day',
 'Reported Licensed Staffing Hours per Resident

In [5]:
cols = [
 'Average Number of Residents Per Day',
 'Provider Resides in Hospital',
 'Most Recent Health Inspection More Than 2 Years Ago',
 'Provider Changed Ownership in Last 12 Months',
 'Total Number of Penalties',
 'Reported Nurse Aide Staffing Hours per Resident per Day',
 'Reported LPN Staffing Hours per Resident per Day',
 'Reported RN Staffing Hours per Resident per Day',
 'Reported Licensed Staffing Hours per Resident per Day',
 'Reported Total Nurse Staffing Hours per Resident per Day',
 'Cited for Abuse'
]

We don't want NaNs in our dataset so we'll filter those out.

In [6]:
snf_data = df[cols].dropna()
snf_data

,Average Number of Residents Per Day,Provider Resides in Hospital,Most Recent Health Inspection More Than 2 Years Ago,Provider Changed Ownership in Last 12 Months,Total Number of Penalties,Reported Nurse Aide Staffing Hours per Resident per Day,Reported LPN Staffing Hours per Resident per Day,Reported RN Staffing Hours per Resident per Day,Reported Licensed Staffing Hours per Resident per Day,Reported Total Nurse Staffing Hours per Resident per Day,Cited for Abuse
Federal Provider Number,,,,,,,,,,,
55249,78.2,False,False,False,0,2.38684,1.06150,0.22008,1.28158,3.66843,False
55305,50.4,False,False,False,0,2.58358,0.82247,0.30496,1.12743,3.71100,False
55341,63.2,False,False,False,0,2.54925,1.24482,0.71746,1.96228,4.51153,False
55612,38.8,False,False,False,2,2.58972,1.11114,0.40391,1.51505,4.10477,True
55653,237.8,False,False,False,0,1.85290,1.17694,0.54549,1.72243,3.57533,False
...,...,...,...,...,...,...,...,...,...,...,...
445472,43.9,False,False,False,0,3.38602,1.52456,0.92610,2.45066,5.83668,False
515185,74.8,False,False,False,1,1.91819,1.16817,0.48395,1.65212,3.57030,False
365081,92.8,False,False,False,1,1.78861,0.94269,0.72234,1.66503,3.45365,False


We'll be passing our dataframe into a separate .csv file after it's prepared, and then using Numpy's loadtxt() function to produce an array of raw data to prepare for our DNN model. Because of this, we want to deal with the Boolean values here and make sure they get stored as numerical representations instead of text, i.e. we want 1s and 0s. We'll explicitly convert them all to integers, then load our data into a separate .csv file:

In [7]:
bool_cols = snf_data.select_dtypes(include=['bool']).columns.to_list()
snf_data[bool_cols] *= 1

In [8]:
snf_data

,Average Number of Residents Per Day,Provider Resides in Hospital,Most Recent Health Inspection More Than 2 Years Ago,Provider Changed Ownership in Last 12 Months,Total Number of Penalties,Reported Nurse Aide Staffing Hours per Resident per Day,Reported LPN Staffing Hours per Resident per Day,Reported RN Staffing Hours per Resident per Day,Reported Licensed Staffing Hours per Resident per Day,Reported Total Nurse Staffing Hours per Resident per Day,Cited for Abuse
Federal Provider Number,,,,,,,,,,,
55249,78.2,0,0,0,0,2.38684,1.06150,0.22008,1.28158,3.66843,0
55305,50.4,0,0,0,0,2.58358,0.82247,0.30496,1.12743,3.71100,0
55341,63.2,0,0,0,0,2.54925,1.24482,0.71746,1.96228,4.51153,0
55612,38.8,0,0,0,2,2.58972,1.11114,0.40391,1.51505,4.10477,1
55653,237.8,0,0,0,0,1.85290,1.17694,0.54549,1.72243,3.57533,0
...,...,...,...,...,...,...,...,...,...,...,...
445472,43.9,0,0,0,0,3.38602,1.52456,0.92610,2.45066,5.83668,0
515185,74.8,0,0,0,1,1.91819,1.16817,0.48395,1.65212,3.57030,0
365081,92.8,0,0,0,1,1.78861,0.94269,0.72234,1.66503,3.45365,0


In [9]:
snf_data.to_csv('NursingHome_data.csv', index=False, header=None)

## Preparing a feed-forward deep neural network model

### Extracting the data

Here, we take our prepared .csv file from the previous steps and load it into two Numpy arrays, one for our features and one for our target, which is the Boolean value for whether a nursing home has been cited for abuse:

In [10]:
snf_data_np = np.loadtxt('NursingHome_data.csv', delimiter=',')

unscaled_inputs_all = snf_data_np[:,1:-1]
targets_all = snf_data_np[:,-1]

In [11]:
unscaled_inputs_all

array([[0.     , 0.     , 0.     , ..., 0.22008, 1.28158, 3.66843],
       [0.     , 0.     , 0.     , ..., 0.30496, 1.12743, 3.711  ],
       [0.     , 0.     , 0.     , ..., 0.71746, 1.96228, 4.51153],
       ...,
       [0.     , 0.     , 0.     , ..., 0.72234, 1.66503, 3.45365],
       [0.     , 0.     , 0.     , ..., 0.60666, 1.58369, 3.75   ],
       [0.     , 0.     , 0.     , ..., 0.26513, 1.50869, 3.23967]])

In [12]:
targets_all

array([0., 0., 0., ..., 0., 0., 0.])

### Balancing the data

When be balance the data for a binary predictive outcome, we're making sure that we have an equal number of 0s and 1s so that our model works on  data it's never seen before. We do this by first getting the sum of the 1s in the target feature, setting a counter to zero, and then setting an empty list to store the indices that we'll take out of our dataset to get this 1:1 ratio:

In [13]:
# balancing the dataset

num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Scaling the data

Next, we need to make sure that our data is intelligible between features. We do this by scaling all our values on a common scale so that they relate to each other by equal weight. We use scikit's preprossing scale() function to do this:

In [14]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffilng the data

It's generally a good idea to shuffle things around so that the model has a "more random" dataset.

In [15]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Splitting the dataset into train, validation, and test sets

Here, we create three separate datasets for our model. We will use the training set to set a baseline for our model to learn, the validation set to measure the training results on a temporary basis, and the test set to validate our model on a permanent basis. This three-way split is important because once we deploy the model on the test set, we can no longer change any parameters or hyperparameters  because it would then be a different model; that's why we need the validation set. We create and validate the splits like so:

In [16]:
# splitting into train, validation, and test
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [17]:
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

588.0 1171 0.502134927412468
75.0 146 0.5136986301369864
69.0 147 0.46938775510204084


### Saving the data into *.npz files

TensorFlow needs data in tensor format to make sense if it. It turns out that .npz files are tensor-friendly, se we'll create those files from our three-way split data:

In [18]:
# saving in tensorflow-friendly format
np.savez('SNF_data_train', inputs=train_inputs, targets=train_targets)
np.savez('SNF_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('SNF_data_test', inputs=test_inputs, targets=test_targets)

### Loading the data back in

Here, we read the .npz files and create the train, validation, and test datasets from them:

In [19]:
npz = np.load('SNF_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('SNF_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('SNF_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Creating the model

Now we finally create our feed-forward DNN. Our input size is the number of features we introduce for the model to learn from; in this case, we have 10 features minus the target. The output size is either a 0 or a 1 for "Cited for Abuse," so the value is 2. The hidden layer size is the number of permutations that the machine goes through to arrive at a prediction of 0 or 1; this can be arbitrary, but in this case we'll go with 50, which is a lot, but this can vary regardless. Next, we create the hidden layers and specify their activation function; in this case, we're choosing the ReLU function, which stands for "rectified linear unit." We understand it to be commonly used and the mathematics behind it are outside the scope of this particular project (though this obviously is important). Finally we compile our model, choosing our optimizer function, here Adam, use sparse categorical cross-entropy for our loss funchion, set our caluclation batch to 100 and instruct the machine to run through 100 epochs, or total cycles to arrive at our final calculations:

In [20]:
input_size = 10                 # number of predictors
output_size = 2                 # only two outputs
hidden_layer_size = 50         # kind of arbitrary? Not sure how to choose this

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

model.fit(train_inputs, train_targets, batch_size=batch_size, epochs=max_epochs, validation_data=(validation_inputs, validation_targets), verbose=2)

Train on 1171 samples, validate on 146 samples
Epoch 1/100
1171/1171 - 1s - loss: 0.6997 - accuracy: 0.5337 - val_loss: 0.6723 - val_accuracy: 0.6096
Epoch 2/100
1171/1171 - 0s - loss: 0.6503 - accuracy: 0.6593 - val_loss: 0.6431 - val_accuracy: 0.6370
Epoch 3/100
1171/1171 - 0s - loss: 0.6221 - accuracy: 0.6781 - val_loss: 0.6334 - val_accuracy: 0.6712
Epoch 4/100
1171/1171 - 0s - loss: 0.6050 - accuracy: 0.6994 - val_loss: 0.6253 - val_accuracy: 0.6918
Epoch 5/100
1171/1171 - 0s - loss: 0.5969 - accuracy: 0.6985 - val_loss: 0.6252 - val_accuracy: 0.6918
Epoch 6/100
1171/1171 - 0s - loss: 0.5935 - accuracy: 0.6960 - val_loss: 0.6235 - val_accuracy: 0.6849
Epoch 7/100
1171/1171 - 0s - loss: 0.5900 - accuracy: 0.7020 - val_loss: 0.6226 - val_accuracy: 0.6849
Epoch 8/100
1171/1171 - 0s - loss: 0.5884 - accuracy: 0.7071 - val_loss: 0.6235 - val_accuracy: 0.6849
Epoch 9/100
1171/1171 - 0s - loss: 0.5896 - accuracy: 0.7020 - val_loss: 0.6307 - val_accuracy: 0.6849
Epoch 10/100
1171/1171 - 0

Epoch 80/100
1171/1171 - 0s - loss: 0.5427 - accuracy: 0.7190 - val_loss: 0.6344 - val_accuracy: 0.6712
Epoch 81/100
1171/1171 - 0s - loss: 0.5441 - accuracy: 0.7199 - val_loss: 0.6292 - val_accuracy: 0.6849
Epoch 82/100
1171/1171 - 0s - loss: 0.5412 - accuracy: 0.7208 - val_loss: 0.6344 - val_accuracy: 0.6712
Epoch 83/100
1171/1171 - 0s - loss: 0.5420 - accuracy: 0.7216 - val_loss: 0.6321 - val_accuracy: 0.6781
Epoch 84/100
1171/1171 - 0s - loss: 0.5410 - accuracy: 0.7199 - val_loss: 0.6373 - val_accuracy: 0.6712
Epoch 85/100
1171/1171 - 0s - loss: 0.5392 - accuracy: 0.7216 - val_loss: 0.6383 - val_accuracy: 0.6575
Epoch 86/100
1171/1171 - 0s - loss: 0.5385 - accuracy: 0.7242 - val_loss: 0.6337 - val_accuracy: 0.6712
Epoch 87/100
1171/1171 - 0s - loss: 0.5375 - accuracy: 0.7233 - val_loss: 0.6376 - val_accuracy: 0.6644
Epoch 88/100
1171/1171 - 0s - loss: 0.5382 - accuracy: 0.7225 - val_loss: 0.6323 - val_accuracy: 0.6849
Epoch 89/100
1171/1171 - 0s - loss: 0.5401 - accuracy: 0.7190 - 

We can see that our model is calculating predictions, but we have a problem: at some point, our val_loss starts increasing while our val_accuracy remains either static or continues to decrease. This means that our model is overfitting the data! It's learning too specifically for this particular problem. Therefore, we need to instruct the machine to stop learning right at the point at which the val_loss stops decreasing. We do this by specifying an early stopping method from Keras:

In [23]:
# Ok, so it's overfit
# need an early stopping mechanism

input_size = 10                
output_size = 2                 
hidden_layer_size = 50         

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=1)

model.fit(train_inputs, train_targets, batch_size=batch_size, epochs=max_epochs, validation_data=(validation_inputs, validation_targets), verbose=2)

Train on 1171 samples, validate on 146 samples
Epoch 1/7
1171/1171 - 0s - loss: 0.7367 - accuracy: 0.4338 - val_loss: 0.6910 - val_accuracy: 0.5479
Epoch 2/7
1171/1171 - 0s - loss: 0.6781 - accuracy: 0.6063 - val_loss: 0.6627 - val_accuracy: 0.6438
Epoch 3/7
1171/1171 - 0s - loss: 0.6431 - accuracy: 0.6815 - val_loss: 0.6433 - val_accuracy: 0.6644
Epoch 4/7
1171/1171 - 0s - loss: 0.6196 - accuracy: 0.6917 - val_loss: 0.6377 - val_accuracy: 0.6781
Epoch 5/7
1171/1171 - 0s - loss: 0.6073 - accuracy: 0.6951 - val_loss: 0.6331 - val_accuracy: 0.6781
Epoch 6/7
1171/1171 - 0s - loss: 0.6011 - accuracy: 0.6934 - val_loss: 0.6356 - val_accuracy: 0.6781
Epoch 7/7
1171/1171 - 0s - loss: 0.5970 - accuracy: 0.6994 - val_loss: 0.6274 - val_accuracy: 0.6712


This is likely to be a much more accurate model, at least as far as this data goes. We observe on our validation set that it seems that out of 10 nursing homes, our model is likely to accurately predict whether 7 of them will be cited for abuse. Not bad.

### Testing the model

If our model is valide, we ought to observe a similar result on our test data, which is data that the model has never seen. Let's see.

In [26]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

147/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [27]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))


Test loss: 0.62. Test accuracy: 65.99%


Our model measures at just about 66% on the test data versus 67% on our validation data. We can expect this model to perform similarly on any data with the same features to predict the same outcome.

## Limitations and Conclusion

Obviously, we've only tested for 10 features here, and there were more than 30 valid features in the original data. We'd have to go through all of them to get a robust idea of all of the information that comes to bear on predicting whether a nursing home will be cited for abuse. Regardless, in this specific context, we can make this prediction using this model at 16 - 17% more accurately than a coin flip. That is statistically significant, not miraculous, but still important. More research will need to be done to determine how this information could be used to improve outcomes for patients and staff in a nursing home setting.

## References

Blankart, C. R., Foster, A. D., & Mor, V. (2019). The effect of political control on financial performance, structure, and outcomes of US nursing homes. Health Services Research, 1. edsbig. https://doi.org/10.1111/1475-6773.13061

Dicken, J. E. (2019). NURSING HOMES: Improved Oversight Needed to Better Protect Residents from Abuse. GAO Reports, 1. pwh.

Dummit, L. A. (2002). Nursing Homes: Quality of Care More Related to Staffing than Spending: GAO-02-431R. GAO Reports, 1.

Grabowski, D. C., Stevenson, D. G., Caudry, D. J., O’Malley, A. J., Green, L. H., Doherty, J. A., & Frank, R. G. (2017). The impact of nursing home pay-for-performance on quality and medicare spending: Results from the nursing home value-based purchasing demonstration. Health Services Research, 4. edsbig. https://doi.org/10.1111/1475-6773.12538

Harrington, C., Schnelle, J. F., McGregor, M., & Simmons, S. F. (2016). The Need for Higher Minimum Staffing Standards in U.S. Nursing Homes. Health Services Insights, 9, 13. edb.

Jane Banaszak-Holl, Whitney B. Berta, Dilys M. Bowman, Joel A. C. Baum, & Will Mitchell. (2002). The Rise of Human Service Chains: Antecedents to Acquisitions and Their Effects on the Quality of Care in US Nursing Homes. Managerial and Decision Economics, 23(4/5), 261. edsjsr.

Konetzka, R. T., Norton, E. C., & Stearns, S. C. (2006). Medicare Payment Changes and Nursing Home Quality: Effects on Long-Stay Residents. International Journal of Health Care Finance and Economics, 6(3), 173. edsjsr. https://doi.org/10.1007/s10754-006-9000-9

Park, Y. J., & Martin, E. G. (2018). Geographic Disparities in Access to Nursing Home Services: Assessing Fiscal Stress and Quality of Care. Health Services Research, 4. edsbig. https://doi.org/10.1111/1475-6773.12801

Trinkoff, A. M., Johantgen, M., Muntaner, C., & Le, R. (2005). Staffing and Worker Injury in Nursing Homes. American Journal of Public Health, 95(7), 1220–1225. pbh.